# First EDA - Team 19

EDA of the followin stores on ReclameAqui (27-03-2020 first 5 pages):
* tokestok-loja-online 
* tokestok 
* madeiramadeira 
* mobly 
* etna-home-store-loja-online 
* etna-home-store 
* casas-bahia-loja-online 
* casas-bahia-lojas-fisicas

## What questions will I be able to answer with this data?
Think about your data in terms of **achieving the objectives** we laid out in previous weeks.  It's also important to **outline insights and possibilities for future action** found in the data.

From our MVP target:
- Focus on answering customers' most frequent questions/problems
- Stores ranking
- Segmented analysis both online vs. bricks and mortar retailers
- Deliver a general snapshot of the vertical

In [1]:
from util import agg_jsonls  # helper function
from pprint import pprint

reviewsRA = agg_jsonls(".")
print(f"We have a total of {len(reviewsRA)} reviews!")
pprint(reviewsRA[0])

We have a total of 282 reviews!
{'business_name': 'Casas Bahia - Loja Online',
 'city': 'Araucária',
 'datetime': '2020-03-27T09:15:00Z',
 'description': 'Fiz a compra no dia 09-03-2020No dia 18-03-2020 recebi um '
                'email afirmando que o produto não poderia ser entregue, sem '
                'dar maiores explicações. Este fato por si só já gerou '
                'frustração. A mensagem afirmava que haveria o estrono do '
                'valor, no entanto a cobrança foi efetuada em minha fatura de '
                'cartão de crédito. Tentei por diversas vezes contato para '
                'resolver e as mensagens são sempre evazivas. Em chat com uma '
                'atendente, fui informado de 1 dia útil para contato, o que '
                'não ocorreu. Anexo algumas imagens.',
 'responses': {'business': [], 'customer': [], 'final': {}},
 'review_ID': '102035475',
 'tags': ['Estorno do valor pago',
          'Caixas de som e fone de ouvido',
          'Informáti

In [2]:
# Get an example of responses > 0
for review in reviewsRA:
    if "responses" in  review and len(review["responses"]["business"]) > 0:
        pprint(review)
        break

{'business_name': '',
 'city': 'São Paulo',
 'datetime': '2020-03-21T13:31:00Z',
 'description': 'Prezados,Ref.- o número do pedido COD 3CACRJ 99- produto - '
                'sofá 3l albany sued areia - data da compra 16/02/2020- data '
                'da suposta entrega 23/03/2020Liguei duas vezes na sexta feira '
                'dia 20 de março para confirmar se receberia o produto que '
                'comprei  - sofá.Não recebi nenhum protocolo de atendimento. '
                'Não tenho o números destes atendimentos. Para a minha '
                'surpresa fui informada no primeiro atendimento que o referido '
                'sofá não seria entregue na data programada. Que a entrega '
                'estaria programada para o dia 30 de março, e não dia 23 de '
                'março conforme informação passada na compra do produto. A '
                'ligação caiu.Na segunda ligação que realizei a atendente me '
                'disse que houve um problema com a minha ent

In [3]:

reviews_with_response = [review for review in reviewsRA if "responses" in  review and len(review["responses"]["business"]) > 0]
reviews_with_no_response = [review for review in reviewsRA if "responses" in  review and len(review["responses"]["business"]) == 0]

print(f"Total of reviews with response {len(reviews_with_response)}")
print(f"Total of reviews with NO response {len(reviews_with_no_response)}")
print(f"We have a total of {len(reviewsRA)} reviews!")

Total of reviews with response 40
Total of reviews with NO response 242
We have a total of 282 reviews!


In [4]:
pprint(reviews_with_response[5])

{'business_name': 'Etna Home Store',
 'city': 'Miracatu',
 'datetime': '2020-03-11T16:54:00Z',
 'description': 'Fiz a compra no site da Etna, dia 27/02/2020, o cupom '
                'funcionou normalmente, era um voucher no valor de 140,00. '
                'Paguei o valor excedente a compra, no mesmo dia através do '
                'boleto. No entanto até hoje, dia 10/03/2020, o status do meu '
                'pedido ainda está Aguardando pagamento. Se não vão entregar o '
                'produto, pelo menos extornem meu dinheiro que paguei a vocês!',
 'responses': {'business': [{'content': 'Maria, boa tarde.Pedimos desculpas '
                                        'pelo ocorrido e salientamos que o '
                                        'nosso objetivo é a satisfação de '
                                        'nossos clientes.Após as devidas '
                                        'considerações em retorno por mensagem '
                                        'privada

In [5]:
from datetime import datetime
import numpy as np
import pandas as pd


def extract_days_to_resolution(review):
    if "datetime" in review and "reply" in review["responses"]["final"]:
        init_dt = datetime.strptime(review["datetime"], '%Y-%m-%dT%H:%M:%SZ')
        final_ans_dt = datetime.strptime(review["responses"]["final"]["reply"][0]["datetime"], '%Y-%m-%dT%H:%M:%SZ')
        return (final_ans_dt - init_dt).days
    return np.nan


def extract_days_to_first_contact(review):
    if "datetime" in review and "business" in review["responses"] and len(review["responses"]["business"]) > 0:
        init_dt = datetime.strptime(review["datetime"], '%Y-%m-%dT%H:%M:%SZ')
        final_ans_dt = datetime.strptime(review["responses"]["business"][0]["datetime"], '%Y-%m-%dT%H:%M:%SZ')
        return (final_ans_dt - init_dt).days
    return np.nan


def format_RA_to_df(review):
    cols_for_df = ['title', 'description', 'business_name', 'uf', 'city', 'review_ID', 'datetime', 'timeCaptured']
    r_cp = {col: review[col] for col in cols_for_df if col in review}
    r_cp["days_to_resolution"] = extract_days_to_resolution(review)
    r_cp["days_to_first_contact"] = extract_days_to_first_contact(review)
    r_cp["resolution_outcome"] = review["responses"]["final"]["result"] if "result" in review["responses"]["final"] else np.nan
    return r_cp
    

dff = pd.DataFrame([format_RA_to_df(row) for row in reviewsRA])
dff.head()


,title,description,business_name,uf,city,review_ID,datetime,timeCaptured,days_to_resolution,days_to_first_contact,resolution_outcome
0,Compra cancelada pelo vendedor - Estorno não e...,Fiz a compra no dia 09-03-2020No dia 18-03-202...,Casas Bahia - Loja Online,PR,Araucária,102035475,2020-03-27T09:15:00Z,2020-03-27T12:16:27Z,NaN,NaN,NaN
1,Pontos TudoAzul não entraram,"Fiz 2 compras de R$ 699,00 cada na promoção de...",Casas Bahia - Loja Online,CE,Itapipoca,102030291,2020-03-26T23:44:00Z,2020-03-27T12:16:27Z,NaN,NaN,NaN
2,Possível [Editado pelo Reclame Aqui],"Olhando stories do meu instagram, até que uma ...",Casas Bahia - Loja Online,RJ,Rio de Janeiro,102030351,2020-03-26T23:47:00Z,2020-03-27T12:16:27Z,NaN,NaN,NaN
3,Fui informada que podia retirar o produto e fu...,Efetuei a compra do liquidificador mondial tur...,Casas Bahia - Loja Online,BA,Salvador,102030361,2020-03-26T23:48:00Z,2020-03-27T12:16:27Z,NaN,NaN,NaN
4,Atraso na Entrega,Eu fiz uma compra pelo site das Casas Bahia no...,Casas Bahia - Loja Online,BA,Salvador,102030421,2020-03-26T23:52:00Z,2020-03-27T12:16:27Z,NaN,NaN,NaN


**Correlation between average response time and whether or not a response was positive/negative.**

We lack more data where we have a "negative" resolution, but for those cases where we have a resolution we see that on average that takes one or two days to close.
We might dig deeper to understand what features these kind of complaints have in common.

In [6]:
dff.resolution_outcome = pd.Categorical(dff.resolution_outcome)
dff.groupby("resolution_outcome").mean()

,days_to_resolution,days_to_first_contact
resolution_outcome,,
positive,1.833333,1.666667


**Average time to company response and evaluating experience.**

On average, companies take 3 days to give the first reply. Also, when aggregating by days before contact, we have a hint that positive responses appear more frequently when the contact is made ealier on.

In [7]:
dff.describe()

,days_to_resolution,days_to_first_contact
count,6.000000,40.000000
mean,1.833333,3.025000
std,2.714160,2.485728
min,0.000000,0.000000
25%,0.000000,1.000000
50%,1.000000,3.000000
75%,2.000000,5.250000
max,7.000000,8.000000


In [8]:
dff.groupby("days_to_first_contact").resolution_outcome.count()

days_to_first_contact
0.0    3
1.0    0
2.0    2
3.0    0
4.0    0
5.0    0
6.0    1
7.0    0
8.0    0
Name: resolution_outcome, dtype: int64

In [16]:
answered_perc = round(dff.days_to_first_contact.count()/dff.shape[0] * 100, 2) 
print(f"Total of answered complaints : {answered_perc}%")

Total of answered complaints : 14.18%


In [21]:
aa = dff.groupby("business_name").count()
aa = round(aa.days_to_first_contact/aa.review_ID * 100, 2) 
pprint(aa)

business_name
                                 20.00
Casas Bahia - Loja Online         0.00
Casas Bahia - Lojas Físicas       0.00
Etna Home Store                  22.22
Etna Home Store - Loja Online    13.04
MadeiraMadeira                    0.00
Mobly                            41.38
Tok&Stok - Loja Física           34.29
Tok&Stok - Loja Online            6.38
dtype: float64


In [7]:
import pandas as pd

pprint(set([i["business_name"] for i in reviews_with_response]))

{'',
 'Etna Home Store',
 'Etna Home Store - Loja Online',
 'Mobly',
 'Tok&Stok - Loja Física',
 'Tok&Stok - Loja Online '}


In [8]:
pprint(set([i["business_name"] for i in reviews_with_no_response]))

{'',
 'Casas Bahia - Loja Online',
 'Casas Bahia - Lojas Físicas',
 'Etna Home Store',
 'Etna Home Store - Loja Online',
 'MadeiraMadeira',
 'Mobly',
 'Tok&Stok - Loja Física',
 'Tok&Stok - Loja Online '}


In [14]:
formatted_rows = []
for review in reviewsRA:
    if "business_name" in review and "responses" in review:
        panda_row = {"business_name": review["business_name"]}
        
        if len(review["responses"]["business"]) > 0:
            panda_row["has_reply"] = True
        else:
            panda_row["has_reply"] = False
            
        if "result" in review["responses"]["final"]:
            panda_row["reply_outcome"] = "positive"
        else:
            panda_row["reply_outcome"] = "negative"
        
        panda_row["tags"] = "_".join(review["tags"])
        
        formatted_rows.append(panda_row)

df = pd.DataFrame(formatted_rows)
df.head()
            


,business_name,has_reply,reply_outcome,tags
0,Casas Bahia - Loja Online,False,negative,Estorno do valor pago_Caixas de som e fone de ...
1,Casas Bahia - Loja Online,False,negative,
2,Casas Bahia - Loja Online,False,negative,Produto não recebido_Panela_Utilidades domésticas
3,Casas Bahia - Loja Online,False,negative,Produto não recebido_Liquidificadores_Eletropo...
4,Casas Bahia - Loja Online,False,negative,Outro problema_Outro Tipo de produto/Serviço_B...


In [13]:
df.groupby(["business_name", "reply_outcome"]).count()

has_reply
business_name                 reply_outcome           
                              negative               9
                              positive               1
Casas Bahia - Loja Online     negative              23
Casas Bahia - Lojas Físicas   negative              36
Etna Home Store               negative              35
                              positive               1
Etna Home Store - Loja Online negative              23
MadeiraMadeira                negative              43
Mobly                         negative              26
                              positive               3
Tok&Stok - Loja Física        negative              35
Tok&Stok - Loja Online        negative              46
                              positive               1

In [17]:
df.loc[df.reply_outcome == "positive"].tags.values

array(['Outro problema_Outro Tipo de produto/Serviço_Não encontrei meu problema',
       'Outro problema_Outro Tipo de produto/Serviço_Móveis Modulados',
       'Móveis em Geral',
       'Produto não recebido_Mesas e Cadeiras_Móveis em Geral',
       'Outro problema_Outro Tipo de produto/Serviço_Marketplace',
       'Produto com defeito_Sofás e Estofados_Móveis em Geral'],
      dtype=object)

In [3]:
unique_tags = set()
for review in reviewsRA:
    if "tags" in review:
        for tag in review["tags"]:
            unique_tags.add(tag)

print(f"We have {len(unique_tags)} unique tags!")
pprint(unique_tags)

We have 89 unique tags!
{'Aromatizantes e Difusores',
 'Assistência técnica e Garantia',
 'Atraso na entrega',
 'Brinquedos e jogos',
 'Caixas de som e fone de ouvido',
 'Cama, Mesa e Banho',
 'Camas e camas-box',
 'Canais de Atendimento',
 'Celular',
 'Celulares e Smartphones',
 'Climatizador de ar',
 'Colchões',
 'Cooktop',
 'Cortina',
 'Decoração',
 'Demora no reparo',
 'Desgaste prematuro',
 'Divergência de valores',
 'Eletrodomésticos',
 'Eletroeletrônicos',
 'Eletroportáteis',
 'Entrega',
 'Equipe de Atendimento',
 'Estofamento rasgado',
 'Estorno do valor pago',
 'Falta de peças',
 'Geladeira-Refrigerador',
 'Iluminação e Elétrica',
 'Informática',
 'Jogo de cama',
 'Lençol',
 'Liquidificadores',
 'Lixeiras',
 'Maquiagem de preço',
 'Marketplace',
 'Mau Atendimento',
 'Mau atendimento no SAC',
 'Mesa quebrada',
 'Mesas e Cadeiras',
 'Montagem',
 'Má qualidade do produto',
 'Móveis',
 'Móveis Modulados',
 'Móveis em Geral',
 'Móveis para Escritório',
 'Móveis para Jardim e Piscin